In [ ]:
###################
#### Prompts   ####
###################

prompts = {}
prompts['single_prompt'] = """
Du hilfst mir dabei Sitzungsprotokolle des Stadtparlaments St Gallen zu verstehen. Verarbeite folgenden Text in diese Überschriften mit den beistehenden Verarbeitunganweißungen. Lasse dir Zeit für jeden Schritt.
[[1]] Zusammenfassung:
Fasse mir den Text zusammen
[[2]] Themen:
Welchem Thema würdest du den Text zuordnen? In einem Wort
[[3]] Links oder Rechts:
Ist der Inhalt eher Links oder Rechts einzuordnen? Gebe mir eine Prozentzahl - Links (0%) - Rechts (100%)
[[4]] Investition:
Hat das Thema zur folge mehr Geld zu investieren oder mehr Geld einzusparen? Antworte in einem Wort?
[[5]] Zeitspanne:
Geht es um eine kurzfristige, mittelfristige oder langfristige Thematik? Antworte in einem Wort?
[[6]] Hauptthemen:
Schreib mir eine liste der Hauptthemen des Textes
[[7]] Intention:
Was ist die Intention des textes?
[[8]] Jahresrelevants:
In welchem Jahr ist der Inhalt des Textes relevant?
[[9]] Zielgruppe:
Welche Zielgruppen der Bevölkerung betrifft der text?
"""
# ...

In [27]:
###################
#### Config    ####
###################

OPENAU_API_KEY = ''
GPT_MODEL = 'gpt-4-1106-preview'

INPUT_FOLDER = 'input'
OUTPUT_FOLDER = 'output'

COLUMNS_OF_INTEREST = [
    'ID',
    'Link zur Sitzung',
    'Legislatur',
    'Jahr',
    'Sitzungsdatum',
    'Download_Sitzung',
    'Download_Protokoll',
    'Person',
    'Partei',
    'Download Traktandumsdokumente',
    'Link zum Traktandum',
    'Aktenplan Ebene5',
    'Traktandentitel',
    'Geschaeft_GUID',
    'Volltext',
    'Quartiere'
]
ID_SOURCE_COLUMN = 'ID'
FILENAME_SOURCE_COLUMN = 'Sitzungsdatum'
COMPLETION_SOURCE_COLUMN = 'Volltext'

In [28]:
###################
#### Functions ####
###################

import datetime
import io
from openai import OpenAI
import os
import simplejson
import pandas as pd

from dotenv import load_dotenv
load_dotenv()


client = OpenAI(
    api_key=OPENAU_API_KEY or os.getenv('OPENAI_API_KEY'),
)

def convert_to_datetime(date_str):
    try:
        if (isinstance(date_str, datetime.datetime)):
            return date_str
        dt = datetime.datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
        return dt
    except:
        return date_str

def convert_datetime_to_str(date):
    try:
        if (isinstance(date, datetime.datetime)):
            return date.strftime("%Y%m%d%H%M%S")
        return date
    except:
        return date

def build_message(prompt, input_text):
    messages = [
        {"role": "user", "content": prompt},
        {"role": "user", "content": f'"{input_text}"'},
    ]
    return messages

def run_completion(prompts, input_text):
    result = {}
    for key, value in prompts.items():
        print(f'Run completion: "{key}"')
        try:
            completion_result = client.chat.completions.create(
                model=GPT_MODEL,
                messages=build_message(value, input_text)
            )
            assistant = completion_result.choices[0].message.content if completion_result else ''
            assistant_result = assistant.strip('"').strip('\'').strip()
            result[key] = assistant_result
        except Exception as e:
            error_message = f'Error occurred while complete "{key}": {str(e)}'
            print(error_message)
    return result

def write_to_output(filename, data):
    current_folder = os.getcwd()
    output_folder_path = os.path.join(current_folder, OUTPUT_FOLDER)
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    file_path = os.path.join(output_folder_path, f'{filename}.json')
    if os.path.exists(file_path):
        os.remove(file_path)

    print(f'Write to file "{filename}"')
    with io.open(file_path, 'w', encoding='utf-8') as outfile:
        simplejson.dump(data, outfile, ensure_ascii=False, default=str, ignore_nan=True)

def read_input_file():
    file_path = None
    current_folder = os.getcwd()
    input_folder = os.path.join(current_folder, INPUT_FOLDER)
    supported_extensions = ['.xlsx', '.xls', '.csv']
    if os.path.exists(input_folder) and os.path.isdir(input_folder):
        for file in os.listdir(input_folder):
            if any(file.endswith(ext) for ext in supported_extensions):
                file_path = os.path.join(input_folder, file)
    if file_path is None:
        raise IOError('Input file not found.')
    
    if file_path.endswith('.xlsx') or file_path.endswith('.xls'):
        df = pd.read_excel(file_path)
    elif file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
    else:
        raise ValueError('Unsupported file type. Please use a .xlsx, .xls or .csv file.')

    df_selected = df[COLUMNS_OF_INTEREST]

    dict_list = df_selected.to_dict(orient='records')

    return dict_list


In [29]:
########################
#### Single Prompt ####
########################
import re


def string_to_dict(input_string):
    try:
        # Split the string into parts using the pattern [[number]]
        parts = re.split(r'\[\[\d+\]\]', input_string)
    
        # Remove empty strings from the list
        parts = [part for part in parts if part]
    
        # Create a dictionary to store the key-value pairs
        result_dict = {}
    
        # Iterate over each part to extract key and value
        for part in parts:
            try:
                # Split the part into key and value using the first colon
                key, value = part.split(':', 1)
        
                # Trim whitespace and add to the dictionary
                result_dict[key.strip('"').strip('\'').strip()] = value.strip('"').strip('\'').strip()
            except:
                pass
    
        return result_dict
    except:
        return {}

In [ ]:
###################
#### Execution ####
###################

print('>>> Start <<<')
input_data = read_input_file()
for data in input_data:
    print('#############################################')
    print(f'Processing meeting date: {data[FILENAME_SOURCE_COLUMN]}')
    print('#############################################')
    result = {}
    result['meta'] = data
    text = data[COMPLETION_SOURCE_COLUMN]
    filename = convert_to_datetime(data[FILENAME_SOURCE_COLUMN])
    if text and text.strip():
        completion = run_completion(prompts, text)
        single_prompt_dict = string_to_dict(completion['single_prompt'])
        result['completion'] = single_prompt_dict
    write_to_output(f'{convert_datetime_to_str(filename)}_{data[ID_SOURCE_COLUMN]}', result)
print('>>> End <<<')


>>> Start <<<
#############################################
Processing meeting date: 2023-12-05 00:00:00
#############################################
Run completion: "single_prompt"
Write to file "20231205000000_1903"
#############################################
Processing meeting date: 2023-12-05 00:00:00
#############################################
Run completion: "single_prompt"
Write to file "20231205000000_1904"
#############################################
Processing meeting date: 2023-11-21 00:00:00
#############################################
Run completion: "single_prompt"
Write to file "20231121000000_1139"
#############################################
Processing meeting date: 2023-11-21 00:00:00
#############################################
Run completion: "single_prompt"
Write to file "20231121000000_1140"
#############################################
Processing meeting date: 2023-11-21 00:00:00
#############################################
Run completion: "single_promp